0

['bb', 'cc']

In [14]:
from selenium import webdriver # 从selenium导入webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import requests

def getip(ipPath,targeturl):
    '''从本地获取代理ip'''
    print('getting ip...')
    with open(ipPath) as f:
        ip_list = f.readlines()
    while True:
        ip = random.choice(ip_list)
        print(ip)
        if checkip(targeturl,ip.strip()):
            proxy = '--proxy-server=http://' + ip.strip()
            return proxy
        else:
            ip_list.remove(ip)
            print('Remove ip:' + ip)

def getheaders():
    user_agent_list = [ \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1" \
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11", \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6", \
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6", \
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1", \
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5", \
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5", \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3", \
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3", \
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3", \
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3", \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3", \
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3", \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3", \
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3", \
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3", \
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24", \
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]
    UserAgent=random.choice(user_agent_list)
    headers = {'User-Agent': UserAgent}
    return headers

def checkip(targeturl,ip):
    headers =getheaders()  # 定制请求头
    proxies = {"http": "http://"+ip, "https": "http://"+ip}  # 代理ip
    print('checking..')
    #proxies = 'http://' + ip.strip()
    response=requests.get(url=targeturl,proxies=proxies,timeout=5).status_code
    '''
    if response == 200 :
        return True
    else:
        return False
    '''
    try:
        response=requests.get(url=targeturl,proxies=proxies,timeout=5).status_code
        if response == 200 :
            print('the ip is working properly')
            return True
        else:
            return False
    except:
        return False
target_url = 'http://kns.cnki.net/kns/brief/result.aspx?dbprefix=SCDB&crossDbcodes=CJFQ,CDFD,CMFD,CPFD,IPFD,CCND,CCJD'
ip_path = 'D:\\codes\\GitProject\\mycodes\\ip.txt'
while_flag = True
while True:
    '''
    opt = webdriver.ChromeOptions()
    opt.add_argument(getip(ip_path,target_url)) #代理
    driver = webdriver.Chrome(chrome_options=opt)
    driver.get(target_url)
    '''
    try:   # 预防代理失效的情况
        opt.add_argument(getip(ip_path,target_url)) #代理
        driver = webdriver.Chrome(chrome_options=opt)
        driver.get(target_url)
    except:
        print('the ip does not work..')
        continue
    time.sleep(7) #代理比较慢
    topic = driver.find_element_by_id('txt_1_value1')
    topic.send_keys(u'财税')
    topic.send_keys(Keys.ENTER)
    time.sleep(2)
    driver.switch_to_frame('iframeResult')#切换frame
    time.sleep(2)
    showPage = driver.find_element_by_xpath('//*[@id="id_grid_display_num"]/a[3]')
    showPage.click() #每页显示50 
    time.sleep(2)
    for i in range(10): #最多选中500
        check_box = driver.find_element_by_id('selectCheckbox')
        check_box.click() # 全选         
        try:
            next_page = driver.find_element_by_link_text(u'下一页')
        except:
            while_flag = False # 标志，以跳出外层while循环，没有下一页
            time.sleep(1)
            break
        next_page.click()
        time.sleep(4)
    # 可以加上一个异常处理模块， 下载页如果出现问题，那么关闭下载页，回到刚刚的页面，重新导出
    export = driver.find_element_by_xpath('//*[@id="J_ORDER"]/tbody/tr[2]/td/table/tbody/tr/td[1]/div/a[2]')
    export.click()
    driver.switch_to_window(driver.window_handles[1]) #可能要重新定位windows
    time.sleep(6)
    zidingyi = driver.find_element_by_xpath('//*[@id="SaveTypeList"]/li[10]/span[1]/a')
    zidingyi.click()  
    time.sleep(4)
    select_all = driver.find_element_by_xpath('//*[@id="selfDefine"]/table/tbody/tr[4]/td/input[1]')
    select_all.click()
    time.sleep(3)
    download = driver.find_element_by_xpath('//*[@id="exportExcel"]')     #下载 excel
    download.click() #能否检测是否下载好了
    time.sleep(12)
    driver.quit()





getting ip...
116.209.55.130:9999

checking..
the ip does not work..
getting ip...
221.4.150.7:8181

checking..
the ip does not work..
getting ip...
116.209.52.231:9999

checking..
the ip does not work..
getting ip...
112.95.23.1:8888

checking..
Remove ip:112.95.23.1:8888

124.79.167.34:9797

checking..
the ip does not work..
getting ip...
218.241.219.226:9999

checking..
the ip does not work..
getting ip...
218.241.219.226:9999

checking..
the ip does not work..
getting ip...
14.115.104.6:808

checking..
the ip does not work..
getting ip...
122.141.74.186:8080

checking..
the ip does not work..
getting ip...
116.209.55.130:9999

checking..
the ip does not work..
getting ip...
221.227.39.160:8118

checking..
Remove ip:221.227.39.160:8118

203.110.164.139:52144

checking..
the ip does not work..
getting ip...
116.209.52.231:9999

checking..
the ip does not work..
getting ip...
119.90.42.20:443

checking..
the ip does not work..
getting ip...
223.156.114.84:9000

checking..
the ip is wo

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="selfDefine"]/table/tbody/tr[4]/td/input[1]"}
  (Session info: chrome=72.0.3626.121)
  (Driver info: chromedriver=72.0.3626.69 (3c16f8a135abc0d4da2dff33804db79b849a7c38),platform=Windows NT 10.0.17134 x86_64)
